In [1]:
import json
import requests
from scrapy import Item, Field, Spider, Selector, Request
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.crawler import CrawlerProcess
from itemadapter import ItemAdapter

In [2]:
class ActorItem(Item):
    bio = Field()
    born = Field()
    movies = Field()
    name = Field()
    url = Field()

    
class MovieItem(Item):
    url = Field()
    title = Field()
    cast = Field()

In [3]:
class CustomWriterPipeline:
    def open_spider(self, spider):
        self.actor_file = open('actors.jl', 'w')
        self.movie_file = open('movies.jl', 'w')

    def close_spider(self, spider):
        self.actor_file.close()
        self.movie_file.close()

    def process_item(self, item, spider):
        line = json.dumps(ItemAdapter(item).asdict()) + "\n"
        if isinstance(item, ActorItem):
            self.actor_file.write(line)
        elif isinstance(item, MovieItem):
            self.movie_file.write(line)
        else:
            return None

In [4]:
class ImdbSpider(CrawlSpider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    base_url = "https://imdb.com"
    start_urls = ['https://www.imdb.com/search/name/?gender=male%2Cfemale&ref_=nv_cel_m',]
    
    custom_settings = {'ITEM_PIPELINES': {'__main__.CustomWriterPipeline': 300,},}
    
    rules = [Rule(LinkExtractor(allow='nm',
                                restrict_css='h3.lister-item-header'),
                  callback='parse_actor'),]
    
    def parse_actor(self, response):
        actor = ActorItem()
        actor['bio'] = (' '
                        .join([text.strip() 
                               for text in (response
                                            .css('#name-bio-text .inline *::text')
                                            .getall())])
                        .split('See full bio')[0]
                        .strip())
        actor['born'] = response.css('#name-born-info *::attr(datetime)').get()
        actor["name"] = (response
                         .css("#name-overview-widget .itemprop::text")
                         .get()
                         .strip())
        actor['url'] = response.url
        actor['movies'] = []
        
        movies = response.xpath('.//*[@id="filmography"]/div[2]/div')
        for movie in movies[:15]:
            actor['movies'].append(movie.css('b a::text').get())
            
            yield response.follow(movie.css('b a::attr(href)').get(),
                                  callback=self.parse_movie)
        
        yield actor

    def parse_movie(self, response):
        movie = MovieItem()
        movie['url'] = response.url
        movie['title'] = response.css('.title_wrapper h1::text').get().strip()
        movie['cast'] = [text.strip() 
                         for text in (response
                                      .css('.cast_list')
                                      .xpath('.//tr/td[2]//a/text()')
                                      .getall())]
        return movie


In [5]:
process = CrawlerProcess()
process.crawl(ImdbSpider)
process.start()

2021-05-19 18:03:30 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-05-19 18:03:31 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.7.3 (default, Mar 27 2019, 22:11:17) - [GCC 7.3.0], pyOpenSSL 19.0.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.7, Platform Linux-4.15.0-74-generic-x86_64-with-debian-buster-sid
2021-05-19 18:03:31 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-05-19 18:03:31 [scrapy.crawler] INFO: Overridden settings:
{}
2021-05-19 18:03:31 [scrapy.extensions.telnet] INFO: Telnet Password: f79ecb16dae8356f
2021-05-19 18:03:31 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-05-19 18:03:31 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.do

2021-05-19 18:03:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/name/nm4331196/> (referer: https://www.imdb.com/search/name/?gender=male%2Cfemale&ref_=nv_cel_m)
2021-05-19 18:03:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/name/nm4502102/>
None
2021-05-19 18:03:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/name/nm0785227/> (referer: https://www.imdb.com/search/name/?gender=male%2Cfemale&ref_=nv_cel_m)
2021-05-19 18:03:34 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.imdb.com/name/nm0015196/> from <GET https://www.imdb.com/name/nm0015196>
2021-05-19 18:03:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/name/nm4331196/>
None
2021-05-19 18:03:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/name/nm0785227/>
None
2021-05-19 18:03:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/name/nm1967827/> (referer: https:/

2021-05-19 18:03:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt2381991/>
None
2021-05-19 18:03:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt4155534/> (referer: https://www.imdb.com/name/nm1289434/)
2021-05-19 18:03:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt6466598/> (referer: https://www.imdb.com/name/nm5939164/)
2021-05-19 18:03:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt4155534/>
None
2021-05-19 18:03:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt6466598/>
None
2021-05-19 18:03:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt2316411/> (referer: https://www.imdb.com/name/nm0491259/)
2021-05-19 18:03:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt4131800/> (referer: https://www.imdb.com/name/nm1289434/)
2021-05-19 18:03:38 [scrapy.core.scraper] DEBUG: Sc

2021-05-19 18:03:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt10394844/> (referer: https://www.imdb.com/name/nm0015196/)
2021-05-19 18:03:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt11092210/> (referer: https://www.imdb.com/name/nm0015196/)
2021-05-19 18:03:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt10944486/>
None
2021-05-19 18:03:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt10394844/>
None
2021-05-19 18:03:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9348714/> (referer: https://www.imdb.com/name/nm0015196/)
2021-05-19 18:03:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9365800/> (referer: https://www.imdb.com/name/nm0015196/)
2021-05-19 18:03:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt11092210/>
None
2021-05-19 18:03:41 [scrapy.core.engine] DEBUG

2021-05-19 18:03:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0870154/>
None
2021-05-19 18:03:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt11138512/>
None
2021-05-19 18:03:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt6017756/>
None
2021-05-19 18:03:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9639470/> (referer: https://www.imdb.com/name/nm5896355/)
2021-05-19 18:03:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt5705956/> (referer: https://www.imdb.com/name/nm2240061/)
2021-05-19 18:03:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt9639470/>
None
2021-05-19 18:03:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt5705956/>
None
2021-05-19 18:03:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt12037194/> (referer: https://www.imdb.co

...

2021-05-19 18:04:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt14402926/>
None
2021-05-19 18:04:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt14324650/> (referer: https://www.imdb.com/name/nm0241049/)
2021-05-19 18:04:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9686790/> (referer: https://www.imdb.com/name/nm0241049/)
2021-05-19 18:04:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0412253/> (referer: https://www.imdb.com/name/nm0241049/)
2021-05-19 18:04:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt14324650/>
None
2021-05-19 18:04:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt8363392/> (referer: https://www.imdb.com/name/nm0241049/)
2021-05-19 18:04:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9853500/> (referer: https://www.imdb.com/name/nm0241049/)
2021-

2021-05-19 18:04:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0458290/> (referer: https://www.imdb.com/name/nm1782667/)
2021-05-19 18:04:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt3310976/> (referer: https://www.imdb.com/name/nm0000478/)
2021-05-19 18:04:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt2630826/>
None
2021-05-19 18:04:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0458290/>
None
2021-05-19 18:04:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt6795022/> (referer: https://www.imdb.com/name/nm0000478/)
2021-05-19 18:04:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt3310976/>
None
2021-05-19 18:04:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt6795022/>
None
2021-05-19 18:04:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/

2021-05-19 18:04:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt2543702/>
None
2021-05-19 18:04:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt3270108/> (referer: https://www.imdb.com/name/nm1602660/)
2021-05-19 18:04:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt3270108/>
None
2021-05-19 18:04:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt3551692/> (referer: https://www.imdb.com/name/nm1602660/)
2021-05-19 18:04:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt4969656/> (referer: https://www.imdb.com/name/nm1602660/)
2021-05-19 18:04:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt3551692/>
None
2021-05-19 18:04:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt3782838/> (referer: https://www.imdb.com/name/nm1602660/)
2021-05-19 18:04:46 [scrapy.core.engine] DEBUG: Cra

2021-05-19 18:04:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt1206328/> (referer: https://www.imdb.com/name/nm0516093/)
2021-05-19 18:04:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0346970/> (referer: https://www.imdb.com/name/nm0516093/)
2021-05-19 18:04:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0274854/>
None
2021-05-19 18:04:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt1206328/>
None
2021-05-19 18:04:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0278920/> (referer: https://www.imdb.com/name/nm0516093/)
2021-05-19 18:04:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0346970/>
None
2021-05-19 18:04:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0297702/> (referer: https://www.imdb.com/name/nm0516093/)
2021-05-19 18:04:49 [scrapy.core.scraper] DEBUG: Sc

2021-05-19 18:04:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0103484/> (referer: https://www.imdb.com/name/nm0005443/)
2021-05-19 18:04:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt7945720/>
None
2021-05-19 18:04:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt0367279/> (referer: https://www.imdb.com/name/nm0005443/)
2021-05-19 18:04:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0103484/>
None
2021-05-19 18:04:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0367279/>
None
2021-05-19 18:04:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt7049682/> (referer: https://www.imdb.com/name/nm0005443/)
2021-05-19 18:04:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt7178640/> (referer: https://www.imdb.com/name/nm0005443/)
2021-05-19 18:04:53 [scrapy.core.scraper] DEBUG: Sc